In [129]:
import pandas as pd 
import numpy as np

In [130]:
df_solar = pd.read_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/HEFTcom24/data/solar1.csv")
df_solar_total = pd.read_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/basic_files/solar_total_production.csv")

In [131]:
df_solar = df_solar.drop(df_solar.columns[0], axis= 1)
df_solar.sort_values(by= "valid_time")
df_solar = df_solar.groupby("valid_time").tail(1).reset_index(drop= True)
df_solar["valid_time"] = pd.to_datetime(df_solar["valid_time"])

In [132]:
df_solar_total.timestamp_utc = pd.to_datetime(df_solar_total.timestamp_utc)

In [133]:
df = pd.merge(df_solar_total, df_solar, left_on='timestamp_utc',right_on='valid_time', how='inner')

In [134]:
mask = (df["reference_time"] >= "2022-11-24") & (df["reference_time"] <= "2022-12-07")

dwd_cols = [
    'CloudCover_Point0_dwd', 'CloudCover_Point1_dwd', 'CloudCover_Point2_dwd',
    'CloudCover_Point3_dwd', 'CloudCover_Point4_dwd', 'CloudCover_Point5_dwd',
    'CloudCover_Point6_dwd', 'CloudCover_Point7_dwd', 'CloudCover_Point8_dwd',
    'CloudCover_Point9_dwd', 'CloudCover_Point10_dwd', 'CloudCover_Point11_dwd',
    'CloudCover_Point12_dwd', 'CloudCover_Point13_dwd', 'CloudCover_Point14_dwd',
    'CloudCover_Point15_dwd', 'CloudCover_Point16_dwd', 'CloudCover_Point17_dwd',
    'CloudCover_Point18_dwd', 'CloudCover_Point19_dwd', 'SolarDownwardRadiation_Point0_dwd',
    'SolarDownwardRadiation_Point1_dwd', 'SolarDownwardRadiation_Point2_dwd',
    'SolarDownwardRadiation_Point3_dwd', 'SolarDownwardRadiation_Point4_dwd',
    'SolarDownwardRadiation_Point5_dwd', 'SolarDownwardRadiation_Point6_dwd',
    'SolarDownwardRadiation_Point7_dwd', 'SolarDownwardRadiation_Point8_dwd',
    'SolarDownwardRadiation_Point9_dwd', 'SolarDownwardRadiation_Point10_dwd',
    'SolarDownwardRadiation_Point11_dwd', 'SolarDownwardRadiation_Point12_dwd',
    'SolarDownwardRadiation_Point13_dwd', 'SolarDownwardRadiation_Point14_dwd',
    'SolarDownwardRadiation_Point15_dwd', 'SolarDownwardRadiation_Point16_dwd',
    'SolarDownwardRadiation_Point17_dwd', 'SolarDownwardRadiation_Point18_dwd',
    'SolarDownwardRadiation_Point19_dwd', 'Temperature_Point0_dwd',
       'Temperature_Point1_dwd', 'Temperature_Point2_dwd',
       'Temperature_Point3_dwd', 'Temperature_Point4_dwd',
       'Temperature_Point5_dwd', 'Temperature_Point6_dwd',
       'Temperature_Point7_dwd', 'Temperature_Point8_dwd',
       'Temperature_Point9_dwd', 'Temperature_Point10_dwd',
       'Temperature_Point11_dwd', 'Temperature_Point12_dwd',
       'Temperature_Point13_dwd', 'Temperature_Point14_dwd',
       'Temperature_Point15_dwd', 'Temperature_Point16_dwd',
       'Temperature_Point17_dwd', 'Temperature_Point18_dwd',
       'Temperature_Point19_dwd', 'Temperature_Point0_ncep',
       'Temperature_Point1_ncep', 'Temperature_Point2_ncep',
       'Temperature_Point3_ncep', 'Temperature_Point4_ncep',
       'Temperature_Point5_ncep', 'Temperature_Point6_ncep',
       'Temperature_Point7_ncep', 'Temperature_Point8_ncep',
       'Temperature_Point9_ncep', 'Temperature_Point10_ncep',
       'Temperature_Point11_ncep', 'Temperature_Point12_ncep',
       'Temperature_Point13_ncep', 'Temperature_Point14_ncep',
       'Temperature_Point15_ncep', 'Temperature_Point16_ncep',
       'Temperature_Point17_ncep', 'Temperature_Point18_ncep',
       'Temperature_Point19_ncep'
]

for dwd_col in dwd_cols:
    ncep_col = dwd_col.replace("dwd", "ncep")
    if ncep_col in df.columns:
        df.loc[mask, dwd_col] = df.loc[mask, ncep_col]

## Features

In [135]:
def pv_temperature_efficiency(irradiance, ambient_temp, NOCT=45, wind_speed=1, eta_0=0.18, beta=0.004):
    """
    Calculate the PV cell temperature and efficiency based on solar irradiance and ambient temperature.
    """
    
    Tc = ambient_temp + (NOCT - 20) * (irradiance / 800)
    
    efficiency = eta_0 * (1 - beta * (Tc - 25))
    
    return Tc, efficiency

In [136]:
temperature_columns = [
    'Temperature_Point0_dwd',
       'Temperature_Point1_dwd', 'Temperature_Point2_dwd',
       'Temperature_Point3_dwd', 'Temperature_Point4_dwd',
       'Temperature_Point5_dwd', 'Temperature_Point6_dwd',
       'Temperature_Point7_dwd', 'Temperature_Point8_dwd',
       'Temperature_Point9_dwd', 'Temperature_Point10_dwd',
       'Temperature_Point11_dwd', 'Temperature_Point12_dwd',
       'Temperature_Point13_dwd', 'Temperature_Point14_dwd',
       'Temperature_Point15_dwd', 'Temperature_Point16_dwd',
       'Temperature_Point17_dwd', 'Temperature_Point18_dwd',
       'Temperature_Point19_dwd', 'Temperature_Point0_ncep',
       'Temperature_Point1_ncep', 'Temperature_Point2_ncep',
       'Temperature_Point3_ncep', 'Temperature_Point4_ncep',
       'Temperature_Point5_ncep', 'Temperature_Point6_ncep',
       'Temperature_Point7_ncep', 'Temperature_Point8_ncep',
       'Temperature_Point9_ncep', 'Temperature_Point10_ncep',
       'Temperature_Point11_ncep', 'Temperature_Point12_ncep',
       'Temperature_Point13_ncep', 'Temperature_Point14_ncep',
       'Temperature_Point15_ncep', 'Temperature_Point16_ncep',
       'Temperature_Point17_ncep', 'Temperature_Point18_ncep',
       'Temperature_Point19_ncep',
]

irradiance_columns = [
    'SolarDownwardRadiation_Point0_dwd', 'SolarDownwardRadiation_Point1_dwd',
    'SolarDownwardRadiation_Point2_dwd', 'SolarDownwardRadiation_Point3_dwd',
    'SolarDownwardRadiation_Point4_dwd', 'SolarDownwardRadiation_Point5_dwd',
    'SolarDownwardRadiation_Point6_dwd', 'SolarDownwardRadiation_Point7_dwd',
    'SolarDownwardRadiation_Point8_dwd', 'SolarDownwardRadiation_Point9_dwd',
    'SolarDownwardRadiation_Point10_dwd', 'SolarDownwardRadiation_Point11_dwd',
    'SolarDownwardRadiation_Point12_dwd', 'SolarDownwardRadiation_Point13_dwd',
    'SolarDownwardRadiation_Point14_dwd', 'SolarDownwardRadiation_Point15_dwd',
    'SolarDownwardRadiation_Point16_dwd', 'SolarDownwardRadiation_Point17_dwd',
    'SolarDownwardRadiation_Point18_dwd', 'SolarDownwardRadiation_Point19_dwd',
    'SolarDownwardRadiation_Point0_ncep', 'SolarDownwardRadiation_Point1_ncep',
    'SolarDownwardRadiation_Point2_ncep', 'SolarDownwardRadiation_Point3_ncep',
    'SolarDownwardRadiation_Point4_ncep', 'SolarDownwardRadiation_Point5_ncep',
    'SolarDownwardRadiation_Point6_ncep', 'SolarDownwardRadiation_Point7_ncep',
    'SolarDownwardRadiation_Point8_ncep', 'SolarDownwardRadiation_Point9_ncep',
    'SolarDownwardRadiation_Point10_ncep', 'SolarDownwardRadiation_Point11_ncep',
    'SolarDownwardRadiation_Point12_ncep', 'SolarDownwardRadiation_Point13_ncep',
    'SolarDownwardRadiation_Point14_ncep', 'SolarDownwardRadiation_Point15_ncep',
    'SolarDownwardRadiation_Point16_ncep', 'SolarDownwardRadiation_Point17_ncep',
    'SolarDownwardRadiation_Point18_ncep', 'SolarDownwardRadiation_Point19_ncep'
]

for i in range(20):
    for source in ['dwd', 'ncep']:
        temp_col = f'Temperature_Point{i}_{source}'
        irradiance_col = f'SolarDownwardRadiation_Point{i}_{source}'
        panel_temp_col = f'Panel_Temperature_Point{i}_{source}'
        panel_eff_col = f'Panel_Efficiency_Point{i}_{source}'
        
        df[panel_temp_col], df[panel_eff_col] = zip(*df.apply(
            lambda row: pv_temperature_efficiency(row[irradiance_col], row[temp_col]), axis=1))

In [137]:
df["SolarDownwardRadiation_Mean"] = df.filter(like= "SolarDownwardRadiation").mean(axis= 1)
df["SolarDownwardRadiation_ncep_Mean"] = df.filter(regex= r"SolarDownwardRadiation.*_ncep").mean(axis= 1)
df["SolarDownwardRadiation_dwd_Mean"] = df.filter(regex= r"SolarDownwardRadiation.*_dwd").mean(axis= 1)
df["SolarDownwardRadiation_RW_Mean_1h"] = df["SolarDownwardRadiation_Mean"].rolling(window= 2).mean()
df["SolarDownwardRadiation_RW_dwd_Mean_30min"] = df["SolarDownwardRadiation_dwd_Mean"].rolling(window= 1).mean()
df["SolarDownwardRadiation_RW_dwd_Mean_1h"] = df["SolarDownwardRadiation_dwd_Mean"].rolling(window= 2).mean()
df["SolarDownwardRadiation_dwd_Mean_Lag_30min"] = df["SolarDownwardRadiation_dwd_Mean"].shift(periods= 1)
df["SolarDownwardRadiation_Mean_Lag_1h"] = df["SolarDownwardRadiation_Mean"].shift(periods= 2)
df["SolarDownwardRadiation_Mean_Lag_24h"] = df["SolarDownwardRadiation_Mean"].shift(periods= 48)
df["Panel_Efficiency_Mean"] = df.filter(regex= r"Panel_Efficiency.*").mean(axis= 1)
df["Panel_Efficiency_Std"] = df.filter(regex= r"Panel_Efficiency.*").std(axis= 1)
df["Panel_Temperature_Mean"] = df.filter(regex= r"Panel_Temperature.*").mean(axis= 1)
df["Panel_Temperature_Std"] = df.filter(regex= r"Panel_Temperature.*").std(axis= 1)
df["Temperature_dwd_Std"] = df.filter(regex= r"Temperature.*_dwd").std(axis= 1)
df["Temperature_dwd_Mean"] = df.filter(regex= r"Temperature.*_dwd").mean(axis= 1)
df["sin_hour"] = np.sin(2 * np.pi * df["valid_time"].dt.hour / 24)
df["cos_hour"] = np.cos(2 * np.pi * df["valid_time"].dt.hour / 24)
df["sin_day"] = np.sin(2 * np.pi * df["valid_time"].dt.dayofyear / 365)
df["cos_day"] = np.cos(2 * np.pi * df["valid_time"].dt.dayofyear / 365)
df["Solar_MWh_Lag_48h"] = df["Solar_MWh_credit"].shift(periods= 96)
df["Capacity_MWP_Lag_48h"] = df["capacity_mwp"].shift(periods= 96)

/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_8450/3364608497.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["sin_day"] = np.sin(2 * np.pi * df["valid_time"].dt.dayofyear / 365)
/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_8450/3364608497.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["cos_day"] = np.cos(2 * np.pi * df["valid_time"].dt.dayofyear / 365)
/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_8450/3364608497.py:20: PerformanceWarning: DataFrame is highly fragmented. 

In [138]:
df = df.sort_values(by= "valid_time")

In [139]:
features = [
    "SolarDownwardRadiation_Mean",
    #"SolarDownwardRadiation_ncep_Mean",
    #"SolarDownwardRadiation_dwd_Mean",
    "SolarDownwardRadiation_RW_Mean_1h",
    "SolarDownwardRadiation_RW_dwd_Mean_30min",
    #"SolarDownwardRadiation_RW_dwd_Mean_1h",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_Mean_Lag_1h",
    "SolarDownwardRadiation_Mean_Lag_24h",
    "Panel_Efficiency_Mean",
    "Panel_Efficiency_Std",
    "Panel_Temperature_Mean",
    "Panel_Temperature_Std",
    "Temperature_dwd_Std",
    "Temperature_dwd_Mean",
    "cos_hour",
    "cos_day",
    "Capacity_MWP_Lag_48h",
    "Solar_MWh_Lag_48h"
    ]

df_test = df[features + ["Solar_MWh_credit"]]

In [140]:
df_test.columns

Index(['SolarDownwardRadiation_Mean', 'SolarDownwardRadiation_RW_Mean_1h',
       'SolarDownwardRadiation_RW_dwd_Mean_30min',
       'SolarDownwardRadiation_dwd_Mean_Lag_30min',
       'SolarDownwardRadiation_Mean_Lag_1h',
       'SolarDownwardRadiation_Mean_Lag_24h', 'Panel_Efficiency_Mean',
       'Panel_Efficiency_Std', 'Panel_Temperature_Mean',
       'Panel_Temperature_Std', 'Temperature_dwd_Std', 'Temperature_dwd_Mean',
       'cos_hour', 'cos_day', 'Capacity_MWP_Lag_48h', 'Solar_MWh_Lag_48h',
       'Solar_MWh_credit'],
      dtype='object')

In [141]:
df_test.to_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/Generation_forecast/Solar_forecast/data/train.csv", index= False)

## Target Norm Capacity

In [142]:
df

,timestamp_utc,generation_mw,installed_capacity_mwp,capacity_mwp,valid_time,reference_time,CloudCover_Point0_dwd,CloudCover_Point1_dwd,CloudCover_Point2_dwd,CloudCover_Point3_dwd,...,Panel_Temperature_Mean,Panel_Temperature_Std,Temperature_dwd_Std,Temperature_dwd_Mean,sin_hour,cos_hour,sin_day,cos_day,Solar_MWh_Lag_48h,Capacity_MWP_Lag_48h
0,2020-09-20 00:00:00+00:00,0.0,2228.460880,2130.749758,2020-09-20 00:00:00+00:00,2020-09-20 00:00:00+00:00,0.426270,0.647109,0.335176,0.652109,...,13.793095,0.978183,0.829341,13.333568,0.000000,1.000000,-0.985948,-0.167052,NaN,NaN
1,2020-09-20 00:30:00+00:00,0.0,2228.460880,2130.749758,2020-09-20 00:30:00+00:00,2020-09-20 00:00:00+00:00,0.435937,0.637988,0.404287,0.652783,...,13.777544,0.971825,0.817136,13.345304,0.000000,1.000000,-0.985948,-0.167052,NaN,NaN
2,2020-09-20 01:00:00+00:00,0.0,2228.460880,2130.749758,2020-09-20 01:00:00+00:00,2020-09-20 00:00:00+00:00,0.445605,0.628867,0.473398,0.653457,...,13.761993,0.967392,0.805767,13.357061,0.258819,0.965926,-0.985948,-0.167052,NaN,NaN
3,2020-09-20 01:30:00+00:00,0.0,2228.460880,2130.749758,2020-09-20 01:30:00+00:00,2020-09-20 00:00:00+00:00,0.486719,0.649297,0.523574,0.652783,...,13.789607,0.932941,0.784640,13.416310,0.258819,0.965926,-0.985948,-0.167052,NaN,NaN
4,2020-09-20 02:00:00+00:00,0.0,2228.460880,2130.749758,2020-09-20 02:00:00+00:00,2020-09-20 00:00:00+00:00,0.527832,0.669727,0.573750,0.652109,...,13.817222,0.902404,0.765711,13.475612,0.500000,0.866025,-0.985948,-0.167052,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63616,2024-05-19 21:30:00+00:00,0.0,2741.831037,2573.542092,2024-05-19 21:30:00+00:00,2024-05-19 00:00:00+00:00,0.000000,0.014082,0.000000,0.022842,...,10.321218,1.131438,0.615667,10.945626,-0.707107,0.707107,0.668064,-0.744104,0.0,2573.662184
63617,2024-05-19 22:00:00+00:00,0.0,2741.831037,2573.542092,2024-05-19 22:00:00+00:00,2024-05-19 00:00:00+00:00,0.000000,0.028164,0.000000,0.045684,...,9.998849,1.105754,0.531990,10.588970,-0.500000,0.866025,0.668064,-0.744104,0.0,2573.662184
63618,2024-05-19 22:30:00+00:00,0.0,2741.831037,2573.542092,2024-05-19 22:30:00+00:00,2024-05-19 00:00:00+00:00,0.000752,0.150234,0.013252,0.124473,...,9.757413,1.087585,0.470686,10.313600,-0.500000,0.866025,0.668064,-0.744104,0.0,2573.662184
63619,2024-05-19 23:00:00+00:00,0.0,2741.831037,2573.542092,2024-05-19 23:00:00+00:00,2024-05-19 00:00:00+00:00,0.001504,0.272305,0.026504,0.203262,...,9.515977,1.081027,0.421697,10.038531,-0.258819,0.965926,0.668064,-0.744104,0.0,2573.662184


In [143]:
import numpy as np

df["Target_Capacity_MWP_%"] = np.where(df["Solar_MWh_credit"] == 0.0, 0, df["Solar_MWh_credit"] / df["capacity_mwp"])
df["Target_Capacity_MWP_%_Lag_48"] = df["Target_Capacity_MWP_%"].shift(96)

In [152]:
df["capacity_mwp"].mean()

np.float64(2281.8743117295026)

In [148]:
features = [
    "SolarDownwardRadiation_Mean",
    "SolarDownwardRadiation_RW_Mean_1h",
    "SolarDownwardRadiation_RW_dwd_Mean_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_Mean_Lag_1h",
    "SolarDownwardRadiation_Mean_Lag_24h",
    "Panel_Efficiency_Mean",
    "Panel_Efficiency_Std",
    "Panel_Temperature_Mean",
    "Panel_Temperature_Std",
    "Temperature_dwd_Std",
    "Temperature_dwd_Mean",
    "cos_hour",
    "cos_day",
    "Capacity_MWP_Lag_48h",
    "Solar_MWh_Lag_48h",
    "Target_Capacity_MWP_%_Lag_48",
    "Target_Capacity_MWP_%"
    ]

df_test_norm = df[features]

In [149]:
df_test_norm

,SolarDownwardRadiation_Mean,SolarDownwardRadiation_RW_Mean_1h,SolarDownwardRadiation_RW_dwd_Mean_30min,SolarDownwardRadiation_dwd_Mean_Lag_30min,SolarDownwardRadiation_Mean_Lag_1h,SolarDownwardRadiation_Mean_Lag_24h,Panel_Efficiency_Mean,Panel_Efficiency_Std,Panel_Temperature_Mean,Panel_Temperature_Std,Temperature_dwd_Std,Temperature_dwd_Mean,cos_hour,cos_day,Capacity_MWP_Lag_48h,Solar_MWh_Lag_48h,Target_Capacity_MWP_%_Lag_48,Target_Capacity_MWP_%
0,0.000000,NaN,0.000000,NaN,NaN,NaN,0.188069,0.000704,13.793095,0.978183,0.829341,13.333568,1.000000,-0.167052,NaN,NaN,NaN,0.0
1,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.188080,0.000700,13.777544,0.971825,0.817136,13.345304,1.000000,-0.167052,NaN,NaN,NaN,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.188091,0.000697,13.761993,0.967392,0.805767,13.357061,0.965926,-0.167052,NaN,NaN,NaN,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.188071,0.000672,13.789607,0.932941,0.784640,13.416310,0.965926,-0.167052,NaN,NaN,NaN,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.188052,0.000650,13.817222,0.902404,0.765711,13.475612,0.866025,-0.167052,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63616,0.005365,0.004906,0.010731,0.008894,-0.001483,-0.000244,0.190569,0.000815,10.321218,1.131438,0.615667,10.945626,0.707107,-0.744104,2573.662184,0.0,0.0,0.0
63617,0.006284,0.005824,0.012567,0.010731,0.004447,-0.000684,0.190801,0.000796,9.998849,1.105754,0.531990,10.588970,0.866025,-0.744104,2573.662184,0.0,0.0,0.0
63618,-0.000911,0.002686,-0.001822,0.012567,0.005365,-0.000098,0.190975,0.000783,9.757413,1.087585,0.470686,10.313600,0.866025,-0.744104,2573.662184,0.0,0.0,0.0
63619,-0.008105,-0.004508,-0.016211,-0.001822,0.006284,0.000488,0.191148,0.000778,9.515977,1.081027,0.421697,10.038531,0.965926,-0.744104,2573.662184,0.0,0.0,0.0


In [150]:
df_test_norm.to_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/Generation_forecast/Solar_forecast/data/train_norm.csv", index= False)